In [ ]:
import random as rd
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

#submit = True
submit = False

In [ ]:
# load the public MNIST dataset.
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)

x_train_norm = x_train/255.
x_test_norm = x_test/255.

In [ ]:
k_train = pd.read_csv('/content/train.csv')
k_labels = np.asarray(k_train['label'])
k_train = k_train.drop(columns=['label'])
k_train_norm = np.asarray(k_train/255.)

val_set_labels = k_labels[-2000:]
val_set_norm = k_train_norm[-2000:]
val_set_norm = val_set_norm.reshape(2000, 28, 28, 1)

k_train_norm = k_train_norm[:-2000]
k_labels = k_labels[:-2000]

In [ ]:
x_data = [row for row in x_test_norm]
x_data += [row for row in x_train_norm]
x_data += [row for row in k_train_norm]
x_data = np.asarray(x_data)
x_data = x_data.reshape((110000, 28, 28, 1))

y_data = [label for label in y_test]
y_data += [label for label in y_train]
y_data += [label for label in k_labels]
y_data = np.asarray(y_data).flatten()

In [ ]:
# train on the training set with some held back for validation #
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=None):

    history = model.fit(x=train_features, y=train_label, 
                        batch_size=batch_size,
                        epochs=epochs, shuffle=True, 
                        validation_split=validation_split,
                        verbose = 1)

    # Gather the model's metrics after each round of training
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist

In [ ]:
 # Set up a CNN with Keras
 
def create_CNN(learning_rate):
    """Create and compile a convolutional neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, 6, activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, 4, activation='relu'))
    model.add(tf.keras.layers.Conv2D(128, 2, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model

In [ ]:
def create_DNN(learning_rate):
    """Create and compile a deep neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Dense(units=784, activation='relu'))  
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model

In [ ]:
def create_mixedNN(learning_rate):
    """Create and compile a deep neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, 6, activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model

In [ ]:
# Train and evalate CNN on ALL the datasets, 
# Train on 10000 + 60000 + 42000 to predict on 28000
learning_rate = 0.001
epochs = 40
batch_size = 500
validation_split = 0.005 # 0.5% off bottom of each array
start_time = time.time()

model3D_A = create_CNN(learning_rate)
model3D_B = create_DNN(learning_rate)
model3D_C = create_CNN(learning_rate)

#x_test_norm:
epochs_A, hist_A = train_model(model3D_A, x_data, y_data, epochs, batch_size, validation_split)
print("\nTrained model A in about {} seconds\n".format((round(time.time()-start_time,4))))
batch_time = time.time()

#x_train_norm:
epochs_B, hist_B = train_model(model3D_B, x_data, y_data, epochs, batch_size, validation_split)
print("\nTrained model B in about {} seconds\n".format((round(time.time()-start_time,4))))
batch_time = time.time()

#k_train_norm:
epochs_C, hist_C = train_model(model3D_C, x_data, y_data, epochs, batch_size, validation_split)
print("\nTrained model C in about {} seconds\n".format((round(time.time()-start_time,4))))

print ("\nTotal time: {} seconds is about {} minutes.".format(round(time.time()-start_time,4),
                                                              (time.time()-start_time)//60))

In [ ]:
model3D_C.evaluate(val_set_norm, val_set_labels,batch_size=100)

In [ ]:
def getPredictions(model=None, dataset=None): 
    ''' Takes a trained model and data
        Returns a List of guesses for that model on that data
    ''' 
    predictions = []
    predicts = model.predict(dataset)
    for j in range(len(dataset)):
        probs = predicts[j] # one row of 10 probabilities 
        max_id = np.argmax(probs)   # index of top probability in row
        predictions += [max_id]
    return predictions

In [25]:
kaggle = pd.read_csv('/content/test.csv')
kaggle_norm = np.asarray(kaggle/255.)
kaggle_norm = kaggle_norm.reshape(28000, 28, 28, 1)

In [36]:
kag_guesses = pd.DataFrame(columns=['A', 'B', 'C', 'AB','AC','CB'])
kag_guesses['A'] = getPredictions(model=model3D_A, dataset=kaggle_norm)
kag_guesses['B'] = getPredictions(model=model3D_B, dataset=kaggle_norm)
kag_guesses['C'] = getPredictions(model=model3D_C, dataset=kaggle_norm)

In [54]:
#%%time
for i in range(len(kag_guesses['A'])):
    kag_guesses['AB'] = kag_guesses['A'].iloc[i] != kag_guesses['B'].iloc[i]
    kag_guesses['AC'] = kag_guesses['A'].iloc[i] != kag_guesses['C'].iloc[i]
    kag_guesses['CB'] = kag_guesses['C'].iloc[i] != kag_guesses['B'].iloc[i]

In [55]:
#All agree on them all!
kag_guesses['CB'].astype(float).sum() + kag_guesses['AC'].astype(float).sum() + kag_guesses['AB'].astype(float).sum()

0.0

In [65]:
if submit:
    kaggles = pd.DataFrame(columns=['ImageId','Label']) 
    kaggles['Label'] = kag_guesses['A']
    kaggles['ImageId'] = [i+1 for i in kaggles.index.values] 
    kaggles.to_csv('submission.csv', columns=["ImageId","Label"], index=False)

##Visualizations

In [ ]:
if not submit:
# Plot a graph of the 'accuracy' metric vs. epochs:
    plt.plot(epochs_A,hist_A["accuracy"])
    plt.plot(epochs_A,hist_A["val_accuracy"])
    plt.show()

    plt.plot(epochs_B,hist_B["accuracy"])
    plt.plot(epochs_B,hist_B["val_accuracy"])
    plt.show()
